#### Script to import necessary data to postgres database
##### MEI 2022/2023

In [ ]:
import sys
!{sys.executable} -m pip install pysftp
!{sys.executable} -m pip install gpxpy
!{sys.executable} -m pip install sqlalchemy-databricks
!{sys.executable} -m pip install psycopg2
!{sys.executable} -m pip install pandas
!{sys.executable} -m pip install os.path2

import sys
import pandas as pd
import csv
import geopandas as gpd
import ipywidgets as widgets
from IPython.display import display
import pysftp
from zipfile import ZipFile
import glob
import gpxpy
from datetime import datetime
import sys
import psycopg2
#import psycopg
import os
import getpass
import subprocess

#!{sys.executable} -m pip install "psycopg[binary]"



#### Connect to existing postgres database
If this database doesn't exist, it will be created

In [ ]:
conn = 0
cursor = 0


def create_postgres_database():
    global conn
    global cursor
    # Get the database connection options
    host = input("Enter the  PostgreSQL host connection: ")
    port = input("Enter the PostgreSQL port connection: ")
    username = input("Enter the PostgreSQL username connection: ")
    password = input("Enter the PostgreSQL password connection: ")
    db_name = input("Enter the PostgreSQL database: ")

    # Connect to the default PostgreSQL database
    conn = psycopg2.connect(
        host=host,
        port=port,
        user=username,
        password=password,
        database="postgres"
    )

    try:
        # Create a new cursor object
        conn.autocommit = True
        cursor = conn.cursor()

        # Check if the database already exists
        cursor.execute("SELECT 1 FROM pg_catalog.pg_database WHERE datname = %s;", (db_name,))
        exists = cursor.fetchone()

        if exists:
            print(f"Database '{db_name}' already exists.")
        else:
            # Create the new database with the user-provided name
            cursor.execute(f"CREATE DATABASE {db_name};")
            print(f"Database '{db_name}' created successfully!")

        # Commit the transaction
        conn.commit()
    except psycopg2.Error as e:
        print("Error creating or checking PostgreSQL database:", e)
    finally:
        # Close the cursor and the connection
        cursor.close()
        conn.close()

        conn = psycopg2.connect(
            host=host,
            port=port,
            user=username,
            password=password,
            database=db_name
        )
        if conn:
            conn.autocommit = True
            cursor = conn.cursor()
            print("Connected to PostgreSQL database")


In [ ]:
create_postgres_database()

#### Import SenseMyCity & Wikiloc data

In [ ]:
# Function to create the recolhas table if it doesn't exist

def create_recolhas_table():
    try:
        cursor.execute('''
            CREATE TABLE IF NOT EXISTS recolhas (
            id  SERIAL PRIMARY KEY,
                id_recolha VARCHAR(255),
                atividade VARCHAR(255),
                id_user VARCHAR(255),
                instante VARCHAR(255),
                latitude FLOAT,
                longitude FLOAT,
                posicao VARCHAR(255),
                modo_transporte VARCHAR(255)
            )
        ''')

        conn.commit()
        print("Table created successfully")
    except Exception as e:
        print("Error occurred during query execution:", e)


In [ ]:
create_recolhas_table()

In [ ]:
#Connecting to the SFTP server and downloading sessions
#Using the pysftp package: https://www.ittsystems.com/how-to-access-sftp-server-in-python/ e https://pysftp.readthedocs.io/en/release_0.2.9/index.html

def connect_to_server():
    dir_src = 'sessions'
    if not os.path.exists(dir_src):
        os.makedirs(dir_src)

   # hostname = input("SFTP server:") #urbysense.dei.uc.pt/
   # username = input("username:") # amistudent
   # password = getpass.getpass("password:") # ami_mei_2023

    hostname = "urbysense.dei.uc.pt"
    username = "amistudent"
    password = "ami_mei_2023"

    with pysftp.Connection(host=hostname, username=username, password=password) as sftp:
        print("Connection successfully established ... ")
        # Switch to a remote directory - change if it is different
        sftp.cwd('data/profile2')

        # Obtain structure of the remote directory 'data/profile2'
        directory_structure = sftp.listdir_attr()

        # Print data
        for attr in directory_structure:
            print(attr.filename, attr)

        #Downloading sessions
        sftp.get_d('.', dir_src, preserve_mtime=True)

In [ ]:
connect_to_server()

In [ ]:
#Function to convert from Hexadecimal Accelerometer data list of values to a Decimal average

def from_hex_to_avg(compressedArray):
    count = 0
    total = 0.0
    for i in range(len(compressedArray) - 3):
        if i % 4 == 0:
            reading = compressedArray[i:i + 4]
            value = int(reading, base=16)
            if value > 128 * 256:
                value = value - 65536
            valuef = value / 256.0
            total += valuef
            count += 1
            print(reading, int(reading, base=16), valuef, total, count)
    return total / count

In [ ]:
from_hex_to_avg(
"001600390031ff78febefe99fef0ff63000f0057001cff51fe5efea1ffd200a900a5ffd4fee4fed7ff91007d00f300c50041ff9cff46ff85fffe0018fffeff47fecbfecafec3fe81fe36fe61fe4cfe2ffeecff57ffb40006003b002b008a00a20049ff6d")

In [12]:
#Unziping when it is needed - SenseMyCity Session
dir_src = 'sessions'
if not os.path.exists(dir_src):
    os.makedirs(dir_src)


# hostname = input("SFTP server:") #urbysense.dei.uc.pt/
# username = input("username:") # amistudent
# password = getpass.getpass("password:") # ami_mei_2023

hostname = "urbysense.dei.uc.pt"
username = "amistudent"
password = "ami_mei_2023"

with pysftp.Connection(host=hostname, username=username, password=password) as sftp:
    print("Connection successfully established ... ")
    # Switch to a remote directory - change if it is different
    sftp.cwd('data/profile2')

    # Obtain structure of the remote directory 'data/profile2'
    directory_structure = sftp.listdir_attr()

    # Print data
    for attr in directory_structure:
        print(attr.filename, attr)

    #Downloading sessions
    sftp.get_d('.', dir_src, preserve_mtime=True)

filenames = os.listdir(dir_src)

if not os.path.exists(dir_src + "_processed"):
    os.makedirs(dir_src + "_processed")

dir_separator = '/'
if sys.platform.startswith('win'):
    dir_separator = '\\'

for file in filenames:
    print(file)
    if os.path.isfile(dir_src + dir_separator + file):
        with open(dir_src + dir_separator + file) as current_file:
            if file.endswith(".zip"):

                tokens = file.split("_")
                session = tokens[1]
                user_id = tokens[2]
                transport = tokens[3]
                activity = '-'.join(tokens[4:]).split(".")[0]

                gps_file = glob.glob("./**/SenseMyCity-" + session + "-gps.csv", recursive=True)
                if len(gps_file) == 0:
                    with ZipFile(dir_src + dir_separator + file, 'r') as zip_ref:
                        zip_ref.extractall(path='SenseMyCity')
                        gps_file = glob.glob("./**/SenseMyCity-" + session + "-gps.csv", recursive=True)

                #Accelerometer sensor as it is (in Hexadecimal values) - do the same for other sensor you want
                #acc_file = glob.glob("./**/SenseMyCity-"+session+"-accelerometer.csv", recursive = True)
                #if len(acc_file) == 0:
                #   with ZipFile( dir_src + "/" + file, 'r') as zip_ref:
                #      zip_ref.extractall()

                recolhas_list = []
                #recolhas_list_acc = []
                #if len(gps_file) > 0 and len(acc_file) > 0 :
                if len(gps_file) > 0:
                    print(gps_file[0])
                    basename = gps_file[0].split(dir_separator)
                    basename_file = basename[len(basename) - 1]
                    #basename = acc_file[0].split("/")
                    #basename_file_acc = basename[len(basename) - 1]

                    if not os.path.exists(dir_src + "_processed" + dir_separator + basename_file.split(".")[0]):
                        print(basename_file)
                        with open(dir_src + "_processed" + dir_separator + basename_file.split(".")[0],
                                  "w") as flag_file:
                            flag_file.write("inserted " + str(len(recolhas_list)) + " points")

                        with open(gps_file[0]) as current_csv_file:
                            for line in current_csv_file:
                                values = line.split(";")
                                if len(values) > 9 and values[0].isnumeric():
                                    recolhas_list.append((values[3], values[7], values[8], values[6]))

                        if len(recolhas_list) > 0:
                            # insert multiple points into the recolhas table
                            sql = "INSERT INTO recolhas(id_recolha, atividade, id_user, instante, latitude, longitude, posicao, modo_transporte) VALUES (" + session + ",'" + activity + "'," + user_id + ",%s,%s,%s,%s,'" + transport + "')"
                            try:
                                # execute the INSERT statements
                                cursor.executemany(sql, recolhas_list)
                                # commit the changes to the database
                                conn.commit()
                            except (Exception, psycopg2.DatabaseError) as error:
                                print(error)

                        #If Accelerometer is also considered in SMC sessions
                        """ with open(acc_file[0]) as current_csv_file:
                            for line in current_csv_file:
                                values = line.split(";")
                                if len(values) > 4 and values[0].isnumeric():
                                    #print(values[2][3:],values[3][3:],values[4][3:])
                                    recolhas_list_acc.append((values[1],str(fromHexToAvg(values[2][3:])),str(fromHexToAvg(values[3][3:])),str(fromHexToAvg(values[4][3:]))))
                                    #print(recolhas_list_acc[len(recolhas_list_acc)-1])
                        if len(recolhas_list_acc) > 0:


                            # insert multiple points into the recolhas_acc table
                            sql = "INSERT INTO recolhas_acc(id_recolha, instante, accx_avg, accy_avg, accz_avg) VALUES (" + session + ",%s,%s,%s,%s)"
                            conn = None
                             try:
                                # execute the INSERT statements
                                cursor.executemany(sql,recolhas_list_acc)
                                # commit the changes to the database
                                conn.commit()
                            except (Exception, psycopg2.DatabaseError) as error:
                                print(error) """


            elif file.endswith(".gpx"):  #Wikiloc session - please see https://pypi.org/project/gpxpy/
                if not os.path.exists(dir_src + "_processed" + dir_separator + file.split(".")[0]):
                    recolhas_list = []
                    with open(dir_src + "_processed" + dir_separator + file.split(".")[0], "w") as flag_file:
                        flag_file.write("inserted " + str(len(recolhas_list)) + " points")
                    with open(dir_src + dir_separator + file) as gpx_file:
                        tokens = file.split("_")
                        session = tokens[1]
                        user_id = tokens[2]
                        transport = tokens[3]
                        activity = '-'.join(tokens[4:]).split(".")[0]
                        gpx = gpxpy.parse(gpx_file)
                        for track in gpx.tracks:
                            for segment in track.segments:
                                for point in segment.points:
                                    recolhas_list.append((point.time.timestamp(), point.latitude, point.longitude,
                                                          point.longitude, point.latitude))

                        if len(recolhas_list) > 0:
                            # insert multiple points into the recolhas table

                            sql = "INSERT INTO recolhas(id_recolha, atividade, id_user, instante, latitude, longitude, posicao, modo_transporte) VALUES (" + session + ",'" + activity + "'," + user_id + ",%s,%s,%s,ST_SetSRID(ST_MakePoint(%s, %s), 4326),'" + transport + "')"
                            try:
                                # execute the INSERT statements
                                cursor.executemany(sql, recolhas_list)
                                # commit the changes to the database
                                conn.commit()
                            except (Exception, psycopg2.DatabaseError) as error:
                                print(error)

Connection successfully established ... 
sensemycity_46166_999_BIKE_HEALTH.zip -rw-r--r--   1 1019     1019       423223 22 Mar 22:53 sensemycity_46166_999_BIKE_HEALTH.zip
sensemycity_46184_596_CAR_SHOPPING_ERRANDS.zip -rw-rw-r--   1 1019     1019      1025497 27 Mar 22:25 sensemycity_46184_596_CAR_SHOPPING_ERRANDS.zip
sensemycity_46197_993_WALK_EDUCATION.zip -rw-rw-r--   1 1019     1019        81785 29 Mar 22:47 sensemycity_46197_993_WALK_EDUCATION.zip
sensemycity_46200_773_CAR_EDUCATION.zip -rw-rw-r--   1 1019     1019      1001805 19 Apr 22:13 sensemycity_46200_773_CAR_EDUCATION.zip
sensemycity_46201_111_WALK_EDUCATION.zip -rw-rw-r--   1 1019     1019       227163 03 Apr 20:57 sensemycity_46201_111_WALK_EDUCATION.zip
sensemycity_46211_993_CAR_ SHOPPING_ERRANDS.zip -rw-rw-r--   1 1019     1019        58100 05 Apr 20:25 sensemycity_46211_993_CAR_ SHOPPING_ERRANDS.zip
sensemycity_46212_993_CAR_SHOPPING_ERRANDS.zip -rw-rw-r--   1 1019     1019        24474 05 Apr 20:25 sensemycity_46212

Exception ignored in: <function Connection.__del__ at 0x0000023019111A60>
Traceback (most recent call last):
  File "C:\Users\isabe\AppData\Roaming\Python\Python39\site-packages\pysftp\__init__.py", line 1013, in __del__
    self.close()
  File "C:\Users\isabe\AppData\Roaming\Python\Python39\site-packages\pysftp\__init__.py", line 784, in close
    if self._sftp_live:
AttributeError: 'Connection' object has no attribute '_sftp_live'


wikiloc_411324_411_WALK_HEALTH.gpx
wikiloc_411325_411_WALK_HEALTH.gpx
wikiloc_411326_411_WALK_HEALTH.gpx
wikiloc_411327_411_WALK_HEALTH.gpx
wikiloc_411328_411_WALK_HEALTH.gpx
wikiloc_411329_411_WALK_HEALTH.gpx
wikiloc_41132_411_WALK_HEALTH.gpx
wikiloc_411330_411_WALK_HEALTH.gpx
wikiloc_411331_411_WALK_HEALTH.gpx
wikiloc_411332_411_WALK_HEALTH.gpx
wikiloc_411333_411_WALK_HEALTH.gpx
wikiloc_411334_411_WALK_HEALTH.gpx
wikiloc_411335_411_WALK_HEALTH.gpx
wikiloc_411336_411_WALK_HEALTH.gpx
wikiloc_411337_411_WALK_HEALTH.gpx
wikiloc_411338_411_WALK_HEALTH.gpx
wikiloc_411339_411_WALK_HEALTH.gpx
wikiloc_41133_411_WALK_HEALTH.gpx
wikiloc_411340_411_WALK_HEALTH.gpx
wikiloc_411341_411_WALK_HEALTH.gpx
wikiloc_411342_411_WALK_HEALTH.gpx
wikiloc_411343_411_WALK_HEALTH.gpx
wikiloc_411344_411_WALK_HEALTH.gpx
wikiloc_411345_411_WALK_HEALTH.gpx
wikiloc_411346_411_WALK_HEALTH.gpx
wikiloc_411347_411_WALK_HEALTH.gpx
wikiloc_411348_411_WALK_HEALTH.gpx
wikiloc_411349_411_WALK_HEALTH.gpx
wikiloc_41134_411_WALK

#### Import FourSquare data

In [ ]:
# Function to create the pois table if it doesn't exist

def create_pois_table():
    try:
        cursor.execute('''
            CREATE TABLE IF NOT EXISTS pois (
            id  SERIAL PRIMARY KEY,
                venueID VARCHAR(255),
                latitude FLOAT,
                longitude FLOAT,
                venueCategory VARCHAR(255),
                countryCode VARCHAR(255)
            )
        ''')

        conn.commit()
        print("Table created successfully")
    except Exception as e:
        print("Error occurred during query execution:", e)

In [ ]:
create_pois_table()

In [ ]:
# File path of the Foursquare POI data
file_name = r'foursquare\dataset_TIST2015_POIs.txt'


def import_foursquare_pois():
    with open(file_name, 'r') as file:
        for line in file:
            row = line.strip().split('	')  # Update the split delimiter to match the blank space
            venueID = row[0]
            latitude = row[1]
            longitude = row[2]
            venueCategory = row[3]
            countryCode = row[4]

            if countryCode == 'PT':  # Add this condition to check if the address is 'PT'
                cursor.execute(
                    "INSERT INTO pois (venueID, latitude, longitude, venueCategory,countryCode) VALUES (%s, %s, %s, %s,%s)",
                    (venueID, latitude, longitude, venueCategory, countryCode)
                )

        conn.commit()




In [ ]:
import_foursquare_pois()

#### Import DGT (Coimbra streets) data

In [14]:
env_dict = os.environ.copy()
print(env_dict['PATH'])

C:\ProgramData\Anaconda3;C:\ProgramData\Anaconda3\Library\mingw-w64\bin;C:\ProgramData\Anaconda3\Library\usr\bin;C:\ProgramData\Anaconda3\Library\bin;C:\ProgramData\Anaconda3\Scripts;C:\ProgramData\Anaconda3\bin;C:\ProgramData\Anaconda3\condabin;C:\Program Files\Common Files\Oracle\Java\javapath;C:\Program Files (x86)\Intel\iCLS Client;C:\Program Files\Intel\iCLS Client;C:\Windows\system32;C:\Windows;C:\Windows\System32\Wbem;C:\Windows\System32\WindowsPowerShell\v1.0;C:\Program Files (x86)\Intel\Intel(R) Management Engine Components\DAL;C:\Program Files\Intel\Intel(R) Management Engine Components\DAL;C:\Program Files (x86)\Intel\Intel(R) Management Engine Components\IPT;C:\Program Files\Intel\Intel(R) Management Engine Components\IPT;C:\Program Files (x86)\NVIDIA Corporation\PhysX\Common;C:\Program Files\Intel\WiFi\bin;C:\Program Files\Common Files\Intel\WirelessCommon;C:\WINDOWS\system32;C:\WINDOWS;C:\WINDOWS\System32\Wbem;C:\WINDOWS\System32\WindowsPowerShell\v1.0;C:\WINDOWS\System32

In [ ]:
#using the command sh2psgql with the following options
# -s SRID -> coordinate reference system id
# -c -> create the table and populate with data
# -I -> creates a spatial index after loading the data
# -S -> simple geometry (instead of MULTI)
# -e -> each insert is executed right after its indication (no batch mode)
#Assuming the following programs (shp2pgsql and psql) are available at PostgreSQL/X/bin in the PATH or specific locations

#creating the SQL output to be imported later into PostgreSQL database
#please verify if environment PATH variable contains PostgreSQL bin folder
env_dict = os.environ.copy()
print(env_dict['PATH'])

completed1 = subprocess.run(['shp2pgsql', '-s', '4326', '-c', '-I', '-S', '-e', 'portugal/roads.shp', 'public.roads'] ,capture_output=True, check=True, env=env_dict)
print('Errors:')
print(completed1.stderr.decode('utf-8'))
print('___________________________\nOutput:')
print(completed1.stdout.decode('utf-8')[:800])
cursor.execute(completed1.stdout.decode('utf-8'))

In [ ]:
completed1 = subprocess.run(['shp2pgsql', '-s', '4326', '-c', '-I', '-S', '-e', 'portugal/landuse.shp', 'public.landuse'] ,capture_output=True, check=True, env=env_dict)
print('Errors:')
print(completed1.stderr.decode('utf-8'))
print('___________________________\nOutput:')
print(completed1.stdout.decode('utf-8')[:800])
cursor.execute(completed1.stdout.decode('utf-8'))

In [ ]:
completed1 = subprocess.run(['shp2pgsql', '-s', '4326', '-c', '-I', '-e', 'canada/trajets_mtl_trajet_2017.shp', 'public.canada'] ,capture_output=True, check=True, env=env_dict)
print('Errors:')
print(completed1.stderr.decode('utf-8'))
print('___________________________\nOutput:')
print(completed1.stdout.decode('utf-8')[:800])
cursor.execute(completed1.stdout.decode('utf-8'))

In [16]:
completed1 = subprocess.run(['shp2pgsql', '-s', '4326', '-c', '-I', '-e', 'portugal/carta_continental/Cont_Freg_CAOP2022.shp', 'public.freguesias'] ,capture_output=True, check=True, env=env_dict)
print('Errors:')
print(completed1.stderr.decode('utf-8'))
print('___________________________\nOutput:')
print(completed1.stdout.decode('utf-8')[:800])
cursor.execute(completed1.stdout.decode('utf-8'))

Errors:
Field area_ha is an FTDouble with width 19 and precision 11
Shapefile type: Polygon
Postgis type: MULTIPOLYGON[2]

___________________________
Output:
SET CLIENT_ENCODING TO UTF8;
SET STANDARD_CONFORMING_STRINGS TO ON;
CREATE TABLE "public"."freguesias" (gid serial,
"dicofre" varchar(6),
"freguesia" varchar(150),
"concelho" varchar(50),
"distrito" varchar(50),
"area_ha" numeric,
"des_simpli" varchar(150));
ALTER TABLE "public"."freguesias" ADD PRIMARY KEY (gid);
SELECT AddGeometryColumn('public','freguesias','geom','4326','MULTIPOLYGON',2);
INSERT INTO "public"."freguesias" ("dicofre","freguesia","concelho","distrito","area_ha","des_simpli",geom) VALUES ('010103','Aguada de Cima','Águeda','Aveiro','2.83931000000e+03','Aguada de Cima','0106000020E6100000010000000103000000010000003B0100000032E6AE49EED5C080627FD91CBCF7400004E78CF4E5D5C080643BDFF0B9F740005C2041EDADD5C000DE0209E8A8F740004FAF94E18ED5C080D85F762B9DF74000713D0A9762D5C
